# Consistent Labeling

The eventual data will be collected from many partner hospitals across Nigeria. If some data is already segmented by a clinician, the numeric labeling of voxels may not be consistent. This script consults the corresponding JSON metadata files for the label mappings and modifies the segmentations such that the labelings are consistent.



In [ ]:
!pip install pydicom SimpleITK

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 16.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
import os
import json
import SimpleITK as sitk
import numpy as np

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
nifti_directory = '/content/drive/MyDrive/NSCLC-Radiomics-Sample/NSCLC-Radiomics-recon/Test Segmentations/Segmentations to be Combined'
json_directory = '/content/drive/MyDrive/NSCLC-Radiomics-Sample/NSCLC-Radiomics-recon/JSON mappings'
output_directory = '/content/drive/MyDrive/NSCLC-Radiomics-Sample/NSCLC-Radiomics-recon/Test Segmentations/Final Segmentations'
output_json_file = '/content/drive/MyDrive/NSCLC-Radiomics-Sample/NSCLC-Radiomics-recon/Test Segmentations/Final Segmentations/combined_label_mapping.json'

In [ ]:
label_priority = {
    "GTV-1": 1,
    "Lung-Left": 2,
    "Lung-Right": 3,
    "Heart": 4,
    "Esophagus": 5,
    "Spinal-Cord": 6
}

def load_nifti_images(directory):
    nifti_images = {}
    for filename in os.listdir(directory):
        if filename.endswith('.nii') or filename.endswith('.nii.gz'):
            base_filename = filename.split('_seg')[0]
            if base_filename not in nifti_images:
                nifti_images[base_filename] = []
            nifti_images[base_filename].append((sitk.ReadImage(os.path.join(directory, filename)), filename))
    return nifti_images

def check_segmentation(nifti_image):
    img_array = sitk.GetArrayFromImage(nifti_image)
    return np.any(img_array > 0)

def extract_labels(nifti_image):
    img_array = sitk.GetArrayFromImage(nifti_image)
    unique_labels = np.unique(img_array)
    return unique_labels

def combine_segmentations(nifti_images, segment_descriptions, label_priority):
    combined_array = None
    priority_array = None

    for nifti_image, description in zip(nifti_images, segment_descriptions):
        img_array = sitk.GetArrayFromImage(nifti_image)
        priority = label_priority.get(description, 99)

        if combined_array is None:
            combined_array = np.zeros_like(img_array)
            priority_array = np.full_like(img_array, 100)

        mask = (img_array > 0) & (priority < priority_array)
        combined_array[mask] = img_array[mask]
        priority_array[mask] = priority

    combined_image = sitk.GetImageFromArray(combined_array)
    combined_image.CopyInformation(nifti_images[0])
    return combined_image

def create_consistent_label_mapping(label_mappings, label_description_mapping):
    consistent_mapping = {}
    uniform_label_description_mapping = {
        1: "Lung-Left",
        2: "Lung-Right"
    }

    for numeric_label, description in label_description_mapping.items():
        if description == "Lung-Left":
            consistent_mapping[numeric_label] = 1
        elif description == "Lung-Right":
            consistent_mapping[numeric_label] = 2

    next_available_label = 3

    for mapping in label_mappings:
        for numeric_label in mapping:
            if numeric_label != 0 and numeric_label not in consistent_mapping:
                if numeric_label in label_description_mapping:
                    if label_description_mapping[numeric_label] not in uniform_label_description_mapping.values():
                        consistent_mapping[numeric_label] = next_available_label
                        uniform_label_description_mapping[next_available_label] = label_description_mapping[numeric_label]
                        next_available_label += 1

    return consistent_mapping, uniform_label_description_mapping

def update_nifti_labels(nifti_image, consistent_mapping):
    img_array = sitk.GetArrayFromImage(nifti_image)
    updated_array = np.zeros_like(img_array)
    for original_label, new_label in consistent_mapping.items():
        updated_array[img_array == original_label] = new_label
    updated_image = sitk.GetImageFromArray(updated_array)
    updated_image.CopyInformation(nifti_image)
    return updated_image

def save_nifti_images(nifti_images, filenames, output_directory):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    for nifti_image, filename in zip(nifti_images, filenames):
        new_filename = f'new_{filename}'
        sitk.WriteImage(nifti_image, os.path.join(output_directory, new_filename))

def load_json_mapping(json_file):
    with open(json_file, 'r') as f:
        label_mapping_data = json.load(f)
    label_mapping_dict = {}
    for segment in label_mapping_data["segmentAttributes"]:
        for attr in segment:
            label_id = attr["labelID"]
            segment_description = attr["SegmentDescription"]
            label_mapping_dict[label_id] = segment_description
    return label_mapping_dict

In [ ]:
nifti_images_dict = load_nifti_images(nifti_directory)
combined_segmentations = {}
label_mappings = []
label_description_mapping = {}

for base_filename, nifti_images in nifti_images_dict.items():
    json_filename = base_filename + '.json'
    json_filepath = os.path.join(json_directory, json_filename)
    if os.path.exists(json_filepath):
        label_mapping_dict = load_json_mapping(json_filepath)
        segment_descriptions = [label_mapping_dict.get(int(extract_labels(img)[0]), "Unknown") for img, _ in nifti_images]
        combined_nifti_image = combine_segmentations([img for img, _ in nifti_images], segment_descriptions, label_priority)
        nifti_images_dict[base_filename] = [combined_nifti_image]

        unique_labels = extract_labels(combined_nifti_image)
        label_mappings.append(unique_labels)
        for numeric_label in unique_labels:
            if numeric_label != 0:  # Assuming 0 is background
                segment_description = label_mapping_dict.get(int(numeric_label))
                if segment_description:
                    label_description_mapping[int(numeric_label)] = segment_description
                    if segment_description not in combined_segmentations:
                        combined_segmentations[segment_description] = []
                    combined_segmentations[segment_description].append(combined_nifti_image)
                else:
                    print(f"  Warning: No description found for Numeric Label: {numeric_label}.")
    else:
        print(f"Warning: No JSON mapping file found for {base_filename}.")

consistent_mapping, uniform_label_description_mapping = create_consistent_label_mapping(label_mappings, label_description_mapping)

with open(output_json_file, 'w') as f:
    json.dump(uniform_label_description_mapping, f, indent=4)

updated_nifti_images = [update_nifti_labels(image[0], consistent_mapping) for image in nifti_images_dict.values()]

save_nifti_images(updated_nifti_images, [f"combined_{base_filename}.nii.gz" for base_filename in nifti_images_dict.keys()], output_directory)

print("Updated NIfTI images and uniform label mapping saved to", output_directory)

Updated NIfTI images and uniform label mapping saved to /content/drive/MyDrive/NSCLC-Radiomics-Sample/NSCLC-Radiomics-recon/Test Segmentations/Final Segmentations
